In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU, ReLU
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop, SGD
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

# Data preparation

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

g = sns.countplot(Y_train)

Y_train.value_counts()

# Check for null and missing values

In [ ]:
X_train.isnull().any().describe()

In [ ]:
test.isnull().any().describe()

# Normalization

In [ ]:
X_train = X_train / 255.0
test = test / 255.0

# Reshape

In [ ]:
X_train_r = X_train.values.reshape(-1,28,28,1)
test_r = test.values.reshape(-1,28,28,1)

# To NumPy

In [ ]:
X_train = X_train.values
Y_train = Y_train.values
test = test.values

# To Categorical

In [ ]:
Y_train_keras = tf.keras.utils.to_categorical(Y_train, num_classes = 10)

# Split training and valdiation

In [ ]:
# This data will be used for deep learning.
X_train_keras, X_val_keras, Y_train_keras, Y_val_keras = train_test_split(X_train_r, Y_train_keras, test_size = 0.1, random_state=0)

In [ ]:
# This data is used for random forests, etc.
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=0)

In [ ]:
plt.imshow(X_train_keras[2])

# Deep Learning by Tensorflow(Keras)

In [ ]:
# model = models.Sequential()

# model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu', input_shape = (28,28,1)))
# model.add(Conv2D(filters = 16, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))

# model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
#                  activation ='relu'))
# model.add(MaxPool2D(pool_size=(2,2)))

# model.add(Flatten())
# model.add(Dense(512, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dense(256, activation = "relu"))
# model.add(BatchNormalization())
# model.add(Dropout(0.1))
# model.add(Dense(10, activation = "softmax"))

In [ ]:
# I referred to the model below　　：　　https://www.kaggle.com/yassineghouzam/introduction-to-cnn-keras-0-997-top-6

model = models.Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Define the optimizer
optimizer = Adam(lr=0.001)

In [ ]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
tf.random.set_seed(0)

history = model.fit(X_train_keras,
                    Y_train_keras,
                    epochs=10,
                    batch_size=128,
                    validation_data=(X_val_keras, Y_val_keras),
                   )

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(hist['epoch'], hist['accuracy'],
           label='Train Accuracy')
  plt.plot(hist['epoch'], hist['val_accuracy'],
           label = 'Val Accuracy')
  plt.ylim([0.7,1.0])
  plt.legend()
  plt.show()
    
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(hist['epoch'], hist['loss'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_loss'],
           label = 'Val Error')
  plt.ylim([0.0,0.3])
  plt.legend()
  plt.show()


plot_history(history)

# Predict_Keras

In [ ]:
pred = model.predict_classes(test_r)

In [ ]:
submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

In [ ]:
submission["Label"] = pred

In [ ]:
submission.to_csv("submission_keras.csv",index=False)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score

In [ ]:
rf = RandomForestClassifier(min_samples_leaf = 2, n_estimators = 1000, random_state = 0, verbose = 1)

In [ ]:
rf.fit(X_train, Y_train)

In [ ]:
rf_pred = rf.predict(X_val)

In [ ]:
print('RandomForest : ' + str(accuracy_score(rf_pred, Y_val)))

 ### RandomForest : accuracy 96%

# Predict_RandomForest

In [ ]:
pred_rf = rf.predict(test)

In [ ]:
submission = pd.read_csv("../input/digit-recognizer/sample_submission.csv")

In [ ]:
submission["Label"] = pred_rf

In [ ]:
submission.to_csv("submission_rf.csv",index=False)